In [2]:
import pandas as pd
HDFStore=pd.HDFStore

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from os import path

import config

num_coefficients=17

%matplotlib inline

In [3]:
#Load data from hdf5 file
rdo_root_path = path.join(config.root_path,config.hw_results_path)
h5file = path.join(rdo_root_path, "archimedean-")
h5filename = h5file + str(num_coefficients) + "-splits.h5"
print(h5filename)
hdf = HDFStore(h5filename)
hdf.keys()

/data/elekin/data/results/handwriting/archimedean-17-splits.h5


['/results/residues/rd/norm/std/window',
 '/results/residues/rd/norm/std/window/x_test',
 '/results/residues/rd/norm/std/window/x_train',
 '/results/residues/rd/norm/std/window/y_test',
 '/results/residues/rd/norm/std/window/y_train',
 '/results/residues/features/norm',
 '/results/residues/features/norm/x_test',
 '/results/residues/features/norm/x_train',
 '/results/residues/features/norm/y_test',
 '/results/residues/features/norm/y_train',
 '/results/radius/r/norm/std/window',
 '/results/radius/r/norm/std/window/x_test',
 '/results/radius/r/norm/std/window/x_train',
 '/results/radius/r/norm/std/window/y_test',
 '/results/radius/r/norm/std/window/y_train',
 '/results/radius/features/norm',
 '/results/radius/features/norm/x_test',
 '/results/radius/features/norm/x_train',
 '/results/radius/features/norm/y_test',
 '/results/radius/features/norm/y_train',
 '/residues/train/features',
 '/residues/train/labels',
 '/residues/test/features',
 '/residues/test/labels']

In [4]:
#Load and scale timeseries between 0 and 1
x_train = hdf["/residues/train/features"].values.astype('float32')
y_train = hdf["/residues/train/labels"].values.astype('int8').reshape(-1,1)
x_test = hdf["/residues/test/features"].values.astype('float32')
y_test = hdf["/residues/test/labels"].values.astype('int8').reshape(-1,1)
hdf.close()

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).take(len(x_train)).batch(config.mini_batch_size).prefetch(2).cache()
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).take(len(x_test)).batch(config.mini_batch_size).prefetch(2).cache()
steps_per_epoch = round(len(train_dataset)/config.mini_batch_size)

if config.print_sample:
    for feat, targ in test_dataset.take(10):
        print ('Features test: {}, Target: {}'.format(feat, targ))

    for feat, targ in test_dataset.take(10):
        print ('Features train: {}, Target: {}'.format(feat, targ))

print("{0} train batches and {1} test batches of {2} mini batch size and {3} steps per epoch".format(len(train_dataset), 
                                                                              len(test_dataset),
                                                                              config.mini_batch_size,
                                                                                steps_per_epoch))
test_dataset

9 train batches and 5 test batches of 4 mini batch size and 2 steps per epoch


<CacheDataset element_spec=(TensorSpec(shape=(None, 4096), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.int8, name=None))>